# Feature Engineering of Weblog data
1. Count of Days of Week where session happened # humans have a more even spread of activity across the week
2. Count of Time of Day ('wee_hours' - 1am-7am , 'morning' - 8am-11am, 'afternoon' - 12pm-3pm, 'evening' - 4pm-7pm, 'night' - 8pm-12am) # more humans are most active between 9am-11am
3. Most frequent site/url
4. Most frequent referral
5. Percentage of each type of method used
6. Percentage of each type of status 
7. Average byte size
8. Duration for the entire session
9. Total number of requests # bots tend to have more requests made
10. Average duration between each request

In [134]:
import pandas as pd
import zipfile
import datetime as dt
import matplotlib.pyplot as plt

In [135]:
zip_path = r'../data/interim/weblog_p1p2.zip'  
extract_folder = r'../data/interim/weblog/'

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

In [136]:
weblog_df = pd.read_csv(extract_folder + 'weblog_p1p2.csv', parse_dates=["datetime"])
weblog_df = weblog_df[weblog_df['Session_ID'] != '-']

In [137]:
def categorize_time(hour):
    if 0 <= hour < 8:
        return "wee_hours"
    elif 8 <= hour < 12:
        return "morning"
    elif 12 <= hour < 16:
        return "afternoon"
    elif 16 <= hour < 19:
        return "evening"
    else:
        return "night"

weblog_df["day_of_week"] = weblog_df["datetime"].dt.day_name()
weblog_df["hour"] = weblog_df["datetime"].dt.hour
weblog_df["time_category"] = weblog_df["hour"].apply(categorize_time)
weblog_df['time_diff'] = weblog_df.groupby('Session_ID')['datetime'].diff().dt.total_seconds()

In [138]:
# Pivot `method` and `status`
method_pivot = pd.crosstab(weblog_df["Session_ID"], weblog_df["method"]).reset_index()
status_pivot = pd.crosstab(weblog_df["Session_ID"], weblog_df["status"]).reset_index()

# Convert counts to percentages for method and status
method_pivot_percent = method_pivot.set_index('Session_ID').div(method_pivot.set_index('Session_ID').sum(axis=1), axis=0) * 100
status_pivot_percent = status_pivot.set_index('Session_ID').div(status_pivot.set_index('Session_ID').sum(axis=1), axis=0) * 100

# Reset index to get Session_ID back as a column
method_pivot_percent = method_pivot_percent.reset_index()
status_pivot_percent = status_pivot_percent.reset_index()

# Add prefix to column name
method_pivot_percent.columns = ['Session_ID'] + [f'method_{col}' for col in method_pivot_percent.columns[1:]] 
status_pivot_percent.columns = ['Session_ID'] + [f'status_{col}' for col in status_pivot_percent.columns[1:]]

# Pivot `day_of_week` and `time_category`
days_pivot = pd.crosstab(weblog_df["Session_ID"], weblog_df["day_of_week"]).reset_index()
days_pivot.columns = ['Session_ID'] + [f'day_{col}' for col in days_pivot.columns[1:]]
time_pivot = pd.crosstab(weblog_df["Session_ID"], weblog_df["time_category"]).reset_index()
time_pivot.columns = ['Session_ID'] + [f'time_{col}' for col in time_pivot.columns[1:]]

# convert days and time columns to binary values instead of counts
# days_pivot.iloc[:, 1:] = (days_pivot.iloc[:, 1:] > 0).astype(int)
# time_pivot.iloc[:, 1:] = (time_pivot.iloc[:, 1:] > 0).astype(int)

test = pd.concat([method_pivot, status_pivot, days_pivot, time_pivot], axis=1)

In [139]:
# Group by Session_ID
session_grouped = weblog_df.groupby(['Session_ID','category']).agg(
    url = ('url', lambda x: x.mode()[0]), # most frequent URL per session
    referrer = ('referrer', lambda x: x.mode()[0]),  # most frequent referrer per session
    avg_byte_size = ('byte_size', 'mean'),  # average byte size per session
    session_duration = ('datetime', lambda x: (x.max() - x.min()).total_seconds()), # total duration per session
    request_count = ('Session_ID', 'size'), # total number of request made per session
    avg_request_duration = ('time_diff', 'mean') # average time interval between each request
).reset_index()

final_df = (
    session_grouped
    .merge(method_pivot_percent, on="Session_ID", how="left")
    .merge(status_pivot_percent, on="Session_ID", how="left")
    .merge(days_pivot, on="Session_ID", how="left")
    .merge(time_pivot, on="Session_ID", how="left")
)

In [140]:
final_df

,Session_ID,category,url,referrer,avg_byte_size,session_duration,request_count,avg_request_duration,method_GET,method_POST,...,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,time_afternoon,time_evening,time_morning,time_night,time_wee_hours
0,01o7p78e2bnu1814jn5k4uqke4,humans,/storage/store_sess_total_mousemv_db.php,https://160.40.52.164/content/computer_securit...,843.406250,31.0,64,0.492063,48.437500,51.562500,...,0,0,0,64,0,64,0,0,0,0
1,03jt2p4bdru20sjb9me2gco6j4,humans,/storage/store_sess_total_mousemv_db.php,http://160.40.52.164/content/web_bots.php,443.734671,882.0,897,0.984375,1.003344,98.996656,...,0,0,0,897,0,0,0,897,0,0
2,0654rvnjhnr0pvsi3qa3e16avo,humans,/storage/store_sess_total_mousemv_db.php,https://160.40.52.164/content/data_management/...,651.146341,112.0,82,1.382716,23.170732,76.829268,...,0,0,0,0,0,82,0,0,0,0
3,06ivkemfgn93qhl5j0vu96rnl4,humans,/storage/store_sess_total_mousemv_db.php,http://160.40.52.164/,599.692308,7.0,13,0.583333,53.846154,46.153846,...,0,0,13,0,0,0,0,13,0,0
4,071tbv7fsev5d64kb0f9jieor6,bots,/storage/store_sess_total_mousemv_db.php,https://160.40.52.164/content/web_bots.php,605.326683,680.0,401,1.700000,11.471322,88.528678,...,0,0,0,0,0,0,401,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,vqdvioip730lq32umqa85ikehl,bots,/storage/store_sess_total_mousemv_db.php,https://160.40.52.164/content/big_data.php,628.471642,537.0,335,1.607784,15.820896,84.179104,...,0,0,335,0,0,0,0,0,0,335
810,vqrt3maidth9lr4df2egocd88g,humans,/storage/store_sess_total_mousemv_db.php,https://160.40.52.164/,947.918919,18.0,37,0.500000,59.459459,40.540541,...,0,0,37,0,0,37,0,0,0,0
811,vtcjrbtjq57mnai4banl61pd25,bots,/storage/store_sess_total_mousemv_db.php,https://160.40.52.164/content/big_data.php,594.541779,638.0,371,1.724324,10.512129,89.487871,...,0,0,0,0,0,129,242,0,0,0
812,vu3fio88psda005g91fbjona0v,bots,/storage/store_sess_total_mousemv_db.php,http://192.168.1.120/content/computer_networks...,552.520833,72.0,96,0.757895,19.791667,80.208333,...,0,0,0,0,96,0,0,96,0,0


In [ ]:
# save file to interim
final_df.to_csv('../data/interim/web_log_features.csv', index=False)